In [2]:
import os
import pandas as pd
import numpy as np
import scipy as sp
from IPython.display import display, HTML
import random
import itertools
import json
import pickle


In [7]:
df = pd.read_csv('./sample-df.csv', sep=',')
display(df.head())

df_no_gold = df[~df['name'].isin(['HIGH','LOW'])]
display(df_no_gold)

,name,statement,claimant,date,originated-from
0,pants-on-fire_doc1,If your child gets this virus their going to h...,Facebook User,2020-25-03,Press Conference
1,pants-on-fire_doc2,We inherited a broken test” for COVID-19.,Donald Trump,2020-30-03,Press Conference
2,pants-on-fire_doc3,Says “there is no” COVID-19 virus.,Facebook User,2020-19-03,Press Conference
3,pants-on-fire_doc4,COVID literally stands for Chinese Originated ...,Facebook User,2020-25-03,Press Conference
4,pants-on-fire_doc5,A post say “hair weave and lace fronts manufac...,Bloggers,2020-26-02,Press Conference


,name,statement,claimant,date,originated-from
0,pants-on-fire_doc1,If your child gets this virus their going to h...,Facebook User,2020-25-03,Press Conference
1,pants-on-fire_doc2,We inherited a broken test” for COVID-19.,Donald Trump,2020-30-03,Press Conference
2,pants-on-fire_doc3,Says “there is no” COVID-19 virus.,Facebook User,2020-19-03,Press Conference
3,pants-on-fire_doc4,COVID literally stands for Chinese Originated ...,Facebook User,2020-25-03,Press Conference
4,pants-on-fire_doc5,A post say “hair weave and lace fronts manufac...,Bloggers,2020-26-02,Press Conference
5,pants-on-fire_doc6,A video says that the Vatican confirmed that P...,Youtube Video,2020-29-02,Press Conference
6,pants-on-fire_doc7,This particular pandemic is one where I don’t ...,Ron Desantis,2020-09-04,Press Conference
7,pants-on-fire_doc8,The government is closing businesses to stop t...,Facebook User,2020-16-03,Press Conference
8,pants-on-fire_doc9,the U.S. is developing an “antivirus” that inc...,Facebook User,2020-16-03,Press Conference
9,pants-on-fire_doc10,Italy arrested a doctor “for intentionally kil...,Bloggers,2020-31-03,Press Conference


In [8]:
'''compute assignemnts'''
hit_len = 3

mat = pd.DataFrame(columns=[f"{x}" for x in range(hit_len)], index=range(int(len(df_no_gold))))

items = set(np.unique(df_no_gold['name']))
items_for_filling = items.copy()

for i_items, item in enumerate(items):
    # insert item and gold
    sampled_items = random.sample(items_for_filling,1)
    #remove from list
    items_for_filling.remove(sampled_items[0])
    
    #add gold and shuffle the order
    sampled_items.append('HIGH')
    sampled_items.append('LOW')
    random.shuffle(sampled_items)
    
    #insert in pandas
    for i_sampled_items ,sampled_item in enumerate(sampled_items):
        mat.at[i_items, str(i_sampled_items)] = sampled_item
    
display(mat)

,0,1,2
0,pants-on-fire_doc10,HIGH,LOW
1,pants-on-fire_doc8,LOW,HIGH
2,HIGH,LOW,pants-on-fire_doc1
3,pants-on-fire_doc5,LOW,HIGH
4,pants-on-fire_doc7,HIGH,LOW
5,pants-on-fire_doc9,HIGH,LOW
6,pants-on-fire_doc4,LOW,HIGH
7,LOW,pants-on-fire_doc3,HIGH
8,LOW,pants-on-fire_doc6,HIGH
9,pants-on-fire_doc2,LOW,HIGH


In [9]:
'''compute units'''
check = mat.copy()

def mapping(element):
    sub = df[df['name']==element]#
    res = {
        "name" : sub['name'].values[0],
        "statement" : sub['statement'].values[0],
        "claimant" : sub['claimant'].values[0],
        "date" : sub['date'].values[0],
        "originated-from" : sub['originated-from'].values[0],
    }
    return res

check = check.applymap(mapping)
display(check)    

,0,1,2
0,"{'name': 'pants-on-fire_doc10', 'statement': '...","{'name': 'HIGH', 'statement': 'Yes, We can!', ...","{'name': 'LOW', 'statement': 'Injecting Bleach..."
1,"{'name': 'pants-on-fire_doc8', 'statement': 'T...","{'name': 'LOW', 'statement': 'Injecting Bleach...","{'name': 'HIGH', 'statement': 'Yes, We can!', ..."
2,"{'name': 'HIGH', 'statement': 'Yes, We can!', ...","{'name': 'LOW', 'statement': 'Injecting Bleach...","{'name': 'pants-on-fire_doc1', 'statement': 'I..."
3,"{'name': 'pants-on-fire_doc5', 'statement': 'A...","{'name': 'LOW', 'statement': 'Injecting Bleach...","{'name': 'HIGH', 'statement': 'Yes, We can!', ..."
4,"{'name': 'pants-on-fire_doc7', 'statement': 'T...","{'name': 'HIGH', 'statement': 'Yes, We can!', ...","{'name': 'LOW', 'statement': 'Injecting Bleach..."
5,"{'name': 'pants-on-fire_doc9', 'statement': 't...","{'name': 'HIGH', 'statement': 'Yes, We can!', ...","{'name': 'LOW', 'statement': 'Injecting Bleach..."
6,"{'name': 'pants-on-fire_doc4', 'statement': 'C...","{'name': 'LOW', 'statement': 'Injecting Bleach...","{'name': 'HIGH', 'statement': 'Yes, We can!', ..."
7,"{'name': 'LOW', 'statement': 'Injecting Bleach...","{'name': 'pants-on-fire_doc3', 'statement': 'S...","{'name': 'HIGH', 'statement': 'Yes, We can!', ..."
8,"{'name': 'LOW', 'statement': 'Injecting Bleach...","{'name': 'pants-on-fire_doc6', 'statement': 'A...","{'name': 'HIGH', 'statement': 'Yes, We can!', ..."
9,"{'name': 'pants-on-fire_doc2', 'statement': 'W...","{'name': 'LOW', 'statement': 'Injecting Bleach...","{'name': 'HIGH', 'statement': 'Yes, We can!', ..."


In [10]:
import string
def randomString(stringLength=10):
    """Generate a random string of fixed length """
    letters = string.ascii_uppercase
    return ''.join(random.choice(letters) for i in range(stringLength))

In [11]:
linear = check.copy()

def mappinglinear(element):
    tin = randomString()
    tout = randomString()
    res = {
        "unit_id":f"unit_{element.name}",
        "token_input": tin,
        "token_output": tout,
        "documents": [x for x in element]
          }
    
    return res

linear = linear.apply(mappinglinear, 1)
linear = [x for x in linear]


display(linear)


[{'unit_id': 'unit_0',
  'token_input': 'QRULPQLLQM',
  'token_output': 'SFHVXDPANH',
  'documents': [{'name': 'pants-on-fire_doc10',
    'statement': 'Italy arrested a doctor “for intentionally killing over 3,000 coronavirus patients.”',
    'claimant': 'Bloggers',
    'date': '2020-31-03',
    'originated-from': 'Press Conference'},
   {'name': 'HIGH',
    'statement': 'Yes, We can!',
    'claimant': 'Barack Obama',
    'date': '2008-01-08',
    'originated-from': 'Facebook Post'},
   {'name': 'LOW',
    'statement': 'Injecting Bleach Could Be a Cure for COVID-19.',
    'claimant': 'Donald Trump',
    'date': '2020-04-23',
    'originated-from': 'Press Conference'}]},
 {'unit_id': 'unit_1',
  'token_input': 'SLTIIPWDZK',
  'token_output': 'VNLMBHCAGY',
  'documents': [{'name': 'pants-on-fire_doc8',
    'statement': 'The government is closing businesses to stop the spread of coronavirus even though "the numbers are nothing compared to H1N1 or Ebola. Everyone needs to realize our gover

In [16]:
'''save the separately the tokens'''

input_tok = [x['token_input'] for x in linear]
output_tok = [x['token_output'] for x in linear]
out = pd.DataFrame()
out['token_input'] = input_tok
out['token_output'] = output_tok

display(out)

,token_input,token_output
0,QRULPQLLQM,SFHVXDPANH
1,SLTIIPWDZK,VNLMBHCAGY
2,LKHMNCVGKA,DRWPGGYDNZ
3,ZURXDRFFQI,XGCATVRYAM
4,BTANGWCXZS,NRDVWGKCBN
5,MTXSANGRXR,YPEYDAFXGL
6,DTDTAXAARJ,SMIXWLAKLP
7,YUQYNJCEJP,KNLLCNGUNP
8,UEOIFMITHY,MYPICLRZJY
9,NNYWEHXNHB,JHRYVOKXEO
